In [3]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_dhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=True, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=2.0, relativistic=True, relax=None)
try:
    assert np.isclose(hf.e_dsrg_mrpt3_1, 0.003323575826786)
except:
    print('Warning: DSRG-MRPT3 energy part 1 is not close to reference value.')
try:
    assert np.isclose(hf.e_dsrg_mrpt2, -0.198886593262808)
except:
    print('Warning: DSRG-MRPT2 energy is not close to reference value.')
try:
    assert np.isclose(hf.e_dsrg_mrpt3_2, -0.004129141541982)
except:
    print('Warning: DSRG-MRPT3 energy part 2 is not close to reference value.')
try:
    assert np.isclose(hf.e_dsrg_mrpt3_3, 0.000009041115140)
except:
    print('Warning: DSRG-MRPT3 energy part 3 is not close to reference value.')

              PySCF DHF interface              
Relativistic DHF Energy:         -99.9638776 Eh
PySCF RHF time:                    0.7512348 s
-----------------------------------------------
Will now allocate 66.724 MB disk space for the AO ERI tensor!

Timing report
....integral transformation:       3.2747862 s
Total time taken:                  3.2752271 s

            Relativistic CASCI(6,8)            
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0047548 s
Time taken for 2-RDM build:        0.0175610 s
Time taken for 3-RDM build:        0.1253285 s
E_frzc:                          -88.8825774 Eh
E_cas:                           -14.2940434 Eh
E_nuc:                             3.1750633 Eh
E_casci:                        -100.0015576 Eh
E0 (from RDM):                  -100.0015576 Eh
Ecorr:                            -0.0376800 Eh

Timing summary
... Hamil build:                    0.0131676 s
... Hamil diag:                    

In [19]:
F_1_tilde = np.zeros(hf.fock.shape, dtype='complex128')
V_1_tilde = np.zeros(hf.dhf_eri_full_asym.shape, dtype='complex128')
F_1_tilde[hf.hole, hf.part] = hf.fock[hf.hole, hf.part].conj() + (hf.fock[hf.hole, hf.part].conj() + np.einsum('xu,iuax,xu->ia', hf.denom_act, hf.T2_1[:,hf.ha,:,hf.pa],hf.cumulants['gamma1']))*hf.d1_exp
V_1_tilde[hf.hole, hf.hole, hf.part, hf.part] = hf.dhf_eri_full_asym[hf.hole, hf.hole, hf.part, hf.part].conj() + (hf.dhf_eri_full_asym[hf.hole, hf.hole, hf.part, hf.part].conj())*hf.d2_exp

F_1_tilde = F_1_tilde.T
V_1_tilde = np.einsum('ijab->abij', V_1_tilde)

In [20]:
fixed_args = (hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)
H_T_C0(None, None, F_1_tilde, V_1_tilde, hf.T1_1, hf.T2_1, *fixed_args)

(-0.07872062351658052+0.01999955146143875j)

In [31]:
F_1_tilde = np.zeros(hf.fock.shape, dtype='complex128')
F_1_tilde[hf.hole,hf.part] = np.copy(hf.fock[hf.hole,hf.part].conj())
F_1_tilde[hf.hole,hf.part] += F_1_tilde[hf.hole,hf.part] * hf.d1_exp
F_1_tilde[hf.hole,hf.part] += np.multiply(hf.d1_exp, np.einsum('xu,iuax,xu->ia',hf.denom_act,hf.T2_1[:,hf.ha,:,hf.pa],hf.cumulants['gamma1']))
F_1_tilde = np.conj(F_1_tilde)

V_1_tilde = np.zeros(hf.dhf_eri_full_asym.shape, dtype='complex128')
V_1_tilde[hf.hole,hf.hole,hf.part,hf.part] = hf.dhf_eri_full_asym[hf.hole,hf.hole,hf.part,hf.part].copy()
V_1_tilde[hf.hole,hf.hole,hf.part,hf.part] += V_1_tilde[hf.hole,hf.hole,hf.part,hf.part] * hf.d2_exp

e_dsrg_mrpt2 = H_T_C0(None, None, F_1_tilde.T, np.einsum('ijab->abij',V_1_tilde), hf.T1_1, hf.T2_1, *fixed_args)
print(e_dsrg_mrpt2)


(-0.1988865932578673-1.613960094032669e-17j)


In [16]:
fixed_args = (hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)

Htilde1_1b = hf.H0A1_1b + 2*hf.F1.conj()
Htilde1_2b = hf.H0A1_2b + 2*hf.dhf_eri_full_asym.conj()

Htilde1A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
Htilde1A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')

H1_T1_C1(Htilde1A1_1b, None, Htilde1_1b, None,          hf.T1_1, None,    *fixed_args)
H1_T2_C1(Htilde1A1_1b, None, Htilde1_1b, None,          None,    hf.T2_1, *fixed_args)
H2_T1_C1(Htilde1A1_1b, None, None, Htilde1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C1(Htilde1A1_1b, None, None, Htilde1_2b, None,    hf.T2_1, *fixed_args)
H1_T2_C2(None, Htilde1A1_2b, Htilde1_1b, None,          None,    hf.T2_1, *fixed_args)
H2_T1_C2(None, Htilde1A1_2b, None, Htilde1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C2(None, Htilde1A1_2b, None, Htilde1_2b, None,    hf.T2_1, *fixed_args)
Htilde1A1_2b = antisymmetrize_and_hermitize(Htilde1A1_2b)# - 0.5*H0A1A1_2b
Htilde1A1_1b = (Htilde1A1_1b + Htilde1A1_1b.T.conj())# - 0.5*H0A1A1_1b
T1_2, T2_2 = hf.form_amplitudes(Htilde1A1_1b, Htilde1A1_2b)
E = 0.5*H_T_C0(None, None, Htilde1_1b, Htilde1_2b, T1_2, T2_2, *fixed_args)
print(f'DSRG-MRPT3 energy pt 2: {E:.15f}')

DSRG-MRPT3 energy pt 2: -0.004129141543266+0.000000000000000j


In [ ]:
hf.e_dsrg_mrpt2 = dsrg_HT(hf.F_1_tilde, hf.V_1_tilde, hf.T1_1, hf.T2_1, hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)
print(hf.e_dsrg_mrpt2)

(-0.19888659325351396+4.084434935566394e-18j)


In [ ]:
H_T_C0(hf.F_1_tilde, hf.V_1_tilde, hf.T1_1, hf.T2_1, hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)

In [ ]:
t0 = time.time()
fixed_args = (hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)

H0A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')
H1_T2_C2(None, H0A1_2b, hf.F0, None, None, hf.T2_1, *fixed_args)
H1_T2_C1(H0A1_1b, None, hf.F0, None, None, hf.T2_1, *fixed_args)
H1_T1_C1(H0A1_1b, None, hf.F0, None, hf.T1_1, None, *fixed_args)
H0A1_2b = antisymmetrize_and_hermitize(H0A1_2b)
H0A1_1b = H0A1_1b + H0A1_1b.T.conj()

H0A1A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A1A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')

H1_T1_C1(H0A1A1_1b, None,      H0A1_1b, None,    hf.T1_1, None,    *fixed_args)
H1_T2_C1(H0A1A1_1b, None,      H0A1_1b, None,    None,    hf.T2_1, *fixed_args)
H2_T1_C1(H0A1A1_1b, None,      None,    H0A1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C1(H0A1A1_1b, None,      None,    H0A1_2b, None,    hf.T2_1, *fixed_args)
H1_T2_C2(None,      H0A1A1_2b, H0A1_1b, None,    None,    hf.T2_1, *fixed_args)
H2_T1_C2(None,      H0A1A1_2b, None,    H0A1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C2(None,      H0A1A1_2b, None,    H0A1_2b, None,    hf.T2_1, *fixed_args)
H0A1A1_2b = antisymmetrize_and_hermitize(H0A1A1_2b)
H0A1A1_1b = H0A1A1_1b + H0A1A1_1b.T.conj()
H0A1A1_1b = -H0A1A1_1b
H0A1A1_2b = -H0A1A1_2b

t1 = time.time()
print("Time for Hbar: ", t1-t0)

E = (1./6) * H_T_C0(None, None, H0A1A1_1b, H0A1A1_2b, hf.T1_1, hf.T2_1, *fixed_args)
print(f'DSRG-MRPT3 energy pt 1: {E:.15f}')


Time for Hbar:  0.509028434753418
DSRG-MRPT3 energy pt 1: 0.003323575826344+0.000000000000000j


In [ ]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf_pt2 = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf_pt2.run_dhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf_pt2.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=True, semi_canonicalize=True)
hf_pt2.run_dsrg_mrpt2(s=2.0, relativistic=True, relax=None)

              PySCF DHF interface              
Relativistic DHF Energy:         -99.9638776 Eh
PySCF RHF time:                    0.8191667 s
-----------------------------------------------
Will now allocate 66.724 MB disk space for the AO ERI tensor!

Timing report
....integral transformation:       3.8645244 s
Total time taken:                  3.8650498 s

            Relativistic CASCI(6,8)            
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0060654 s
Time taken for 2-RDM build:        0.0176573 s
Time taken for 3-RDM build:        0.1317499 s
E_frzc:                          -88.8825774 Eh
E_cas:                           -14.2940434 Eh
E_nuc:                             3.1750633 Eh
E_casci:                        -100.0015576 Eh
E0 (from RDM):                  -100.0015576 Eh
Ecorr:                            -0.0376800 Eh

Timing summary
... Hamil build:                    0.0127583 s
... Hamil diag:                    

In [ ]:
hf.T2_1 = hf_pt2.T2_1
hf.T1_1 = hf_pt2.T1_1
hf.cumulants = hf_pt2.cumulants

In [ ]:
hf.compute_energy_pt3_1()

DSRG-MRPT3 energy pt 1: 0.002632797241654-0.000000000000000j
Time taken:  0.47235846519470215


In [ ]:
fixed_args = (hf_pt2.cumulants['gamma1'], hf_pt2.cumulants['eta1'], hf_pt2.cumulants['lambda2'], hf_pt2.cumulants['lambda3'], hf_pt2)

H0A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')
compute_ht_h1_only(fixed_args, H0A1_1b, H0A1_2b, hf.F0, hf_pt2.T1_1, hf_pt2.T2_1)

H0A1A1_1b = np.zeros((hf_pt2.nlrg,hf_pt2.nlrg), dtype='complex128')
H0A1A1_2b = np.zeros((hf_pt2.nlrg,hf_pt2.nlrg,hf_pt2.nlrg,hf_pt2.nlrg), dtype='complex128')

H1_T1_C1(H0A1A1_1b, None,      H0A1_1b, None,    hf_pt2.T1_1, None,    *fixed_args)
H1_T2_C1(H0A1A1_1b, None,      H0A1_1b, None,    None,    hf_pt2.T2_1, *fixed_args)
H2_T1_C1(H0A1A1_1b, None,      None,    H0A1_2b, hf_pt2.T1_1, None,    *fixed_args)
H2_T2_C1(H0A1A1_1b, None,      None,    H0A1_2b, None,    hf_pt2.T2_1, *fixed_args)
H1_T2_C2(None,      H0A1A1_2b, H0A1_1b, None,    None,    hf_pt2.T2_1, *fixed_args)
H2_T1_C2(None,      H0A1A1_2b, None,    H0A1_2b, hf_pt2.T1_1, None,    *fixed_args)
H2_T2_C2(None,      H0A1A1_2b, None,    H0A1_2b, None,    hf_pt2.T2_1, *fixed_args)
H0A1A1_2b = antisymmetrize_and_hermitize(H0A1A1_2b)
H0A1A1_1b = H0A1A1_1b + H0A1A1_1b.T.conj()
H0A1A1_1b = -H0A1A1_1b
H0A1A1_2b = -H0A1A1_2b


E = (1./6) * H_T_C0(None, None, H0A1A1_1b, H0A1A1_2b, hf_pt2.T1_1, hf_pt2.T2_1, *fixed_args)
print(f'DSRG-MRPT3 energy pt 1: {E:.15f}')

DSRG-MRPT3 energy pt 1: 0.003265049671309-0.000000000000000j


In [1]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
hf.run_dsrg_mrpt2(s=1.0, relativistic=False, relax='iterate')

              PySCF RHF interface              
Non-relativistic RHF Energy:     -99.8728525 Eh
PySCF RHF time:                    0.4194305 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0246420 s
-----------------------------------------------
RHF time:                          0.4441302 s

          Non-relativistic CASCI(6,8)          
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0073361 s
Time taken for 2-RDM build:        0.0247209 s
Time taken for 3-RDM build:        0.1641724 s
E_frzc:                          -88.7919897 Eh
E_cas:                           -14.2934663 Eh
E_nuc:                             3.1750633 Eh
E_casci:                         -99.9103927 Eh
E0 (from RDM):                   -99.9103927 Eh
Ecorr:                            -0.0375403 Eh
Error to PySCF:                    0.0000000 Eh

Timing summary
... Hamil build:                    0.0187

In [2]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=1.0, relativistic=False, relax='iterate', pt2_only=True)

              PySCF RHF interface              
Non-relativistic RHF Energy:     -99.8728525 Eh
PySCF RHF time:                    0.3390441 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0223668 s
-----------------------------------------------
RHF time:                          0.3614709 s

          Non-relativistic CASCI(6,8)          
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0064788 s
Time taken for 2-RDM build:        0.0251486 s
Time taken for 3-RDM build:        0.1663568 s
E_frzc:                          -88.7919897 Eh
E_cas:                           -14.2934663 Eh
E_nuc:                             3.1750633 Eh
E_casci:                         -99.9103927 Eh
E0 (from RDM):                   -99.9103927 Eh
Ecorr:                            -0.0375403 Eh
Error to PySCF:                   -0.0000000 Eh

Timing summary
... Hamil build:                    0.0195

In [1]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=1.0, relativistic=False, relax=3, pt2_only=False)

              PySCF RHF interface              
Non-relativistic RHF Energy:     -99.8728525 Eh
PySCF RHF time:                    0.4111364 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0265958 s
-----------------------------------------------
RHF time:                          0.4377947 s

          Non-relativistic CASCI(6,8)          
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0063989 s
Time taken for 2-RDM build:        0.0249004 s
Time taken for 3-RDM build:        0.1670895 s
E_frzc:                          -88.7919897 Eh
E_cas:                           -14.2934663 Eh
E_nuc:                             3.1750633 Eh
E_casci:                         -99.9103927 Eh
E0 (from RDM):                   -99.9103927 Eh
Ecorr:                            -0.0375403 Eh
Error to PySCF:                    0.0000000 Eh

Timing summary
... Hamil build:                    0.0184

In [2]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=1.0, relativistic=False, relax='iterate', pt2_only=False)

              PySCF RHF interface              
Non-relativistic RHF Energy:     -99.8728525 Eh
PySCF RHF time:                    0.3768010 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0238726 s
-----------------------------------------------
RHF time:                          0.4007311 s

          Non-relativistic CASCI(6,8)          
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0063190 s
Time taken for 2-RDM build:        0.0264106 s
Time taken for 3-RDM build:        0.1565399 s
E_frzc:                          -88.7919897 Eh
E_cas:                           -14.2934663 Eh
E_nuc:                             3.1750633 Eh
E_casci:                         -99.9103927 Eh
E0 (from RDM):                   -99.9103927 Eh
Ecorr:                            -0.0375403 Eh
Error to PySCF:                    0.0000000 Eh

Timing summary
... Hamil build:                    0.0190

In [17]:
print(f'{hf.e_casci=}')
print(f'{hf.e_dsrg_mrpt2=}')
print(f'{hf.e_dsrg_mrpt3_1=}')
print(f'{hf.e_dsrg_mrpt3_2=}')
print(f'{hf.e_dsrg_mrpt3_3=}')
print(f'{hf.e_dsrg_mrpt3=}')

hf.e_casci=-99.90910395370764
hf.e_dsrg_mrpt2=(-0.19937544936736556+0j)
hf.e_dsrg_mrpt3_1=(0.0025254255329648038-0j)
hf.e_dsrg_mrpt3_2=(-0.005694622162708253+0j)
hf.e_dsrg_mrpt3_3=(-0.00013682821203142792+0j)
hf.e_dsrg_mrpt3=-100.11178542791679


In [4]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_dhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=True, semi_canonicalize=True)
hf.run_dsrg_mrpt3(s=1.0, relativistic=True, relax='iterate', pt2_only=False)

              PySCF DHF interface              
Relativistic DHF Energy:         -99.9638776 Eh
PySCF RHF time:                    0.7047310 s
-----------------------------------------------
Will now allocate 66.724 MB disk space for the AO ERI tensor!

Timing report
....integral transformation:       3.3019493 s
Total time taken:                  3.3024664 s

            Relativistic CASCI(6,8)            
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0048668 s
Time taken for 2-RDM build:        0.0220795 s
Time taken for 3-RDM build:        0.1167464 s
E_frzc:                          -88.8825774 Eh
E_cas:                           -14.2940434 Eh
E_nuc:                             3.1750633 Eh
E_casci:                        -100.0015576 Eh
E0 (from RDM):                  -100.0015576 Eh
Ecorr:                            -0.0376800 Eh

Timing summary
... Hamil build:                    0.0126553 s
... Hamil diag:                    

In [3]:
from relforte import *
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_dhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=True, semi_canonicalize=True)
hf.run_dsrg_mrpt2(s=1.0, relativistic=True, relax='iterate')

              PySCF DHF interface              
Relativistic DHF Energy:         -99.9638776 Eh
PySCF RHF time:                    0.7299006 s
-----------------------------------------------
Will now allocate 66.724 MB disk space for the AO ERI tensor!

Timing report
....integral transformation:       3.3964188 s
Total time taken:                  3.3969440 s

            Relativistic CASCI(6,8)            
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0042455 s
Time taken for 2-RDM build:        0.0167527 s
Time taken for 3-RDM build:        0.1377902 s
E_frzc:                          -88.8825774 Eh
E_cas:                           -14.2940434 Eh
E_nuc:                             3.1750633 Eh
E_casci:                        -100.0015576 Eh
E0 (from RDM):                  -100.0015576 Eh
Ecorr:                            -0.0376800 Eh

Timing summary
... Hamil build:                    0.0128052 s
... Hamil diag:                    

In [49]:
from relforte import *
import pyscf.cc

def eigh_gen(A, S, eta=1e-9):
    sevals, sevecs = np.linalg.eigh(S)
    trunc_indices = np.where(np.abs(sevals) > eta)[0]
    X = sevecs[:,trunc_indices] / np.sqrt(sevals[trunc_indices])
    Ap = X.T @ A @ X
    return np.linalg.eigh(Ap)

In [59]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
O
H 1 1.00
H 1 1.00 2 103.1
''',
    basis = '6-31g', spin=4, charge=0, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
# print(mf.e_tot)
mc = pyscf.mcscf.CASSCF(mf, 6, 8)
mc.kernel()

-75.34185452312367


(-75.34251422898117,
 -22.947397629920772,
 FCIvector([[-9.99808228e-01,  3.41025703e-15, -2.25742892e-04,
             -9.25607172e-16, -2.38691513e-17,  5.62329787e-18,
              5.16956547e-16, -9.99993675e-04, -4.05164723e-16,
              9.06417983e-18,  1.77474456e-03, -1.94225121e-16,
             -1.51633049e-02, -1.23036455e-18,  1.22221480e-02]]),
 array([[ 9.92524712e-01, -2.37086978e-01,  8.95258255e-16,
          6.34093266e-02,  9.40606111e-18, -8.77933242e-02,
          2.37637281e-15, -2.92830050e-02,  1.54278812e-15,
         -1.42224401e-15,  6.79164580e-02,  2.85663648e-15,
          3.60419955e-02],
        [ 2.96106520e-02,  5.38094508e-01, -2.50601712e-16,
         -1.71248672e-01, -4.63457651e-17,  2.26519388e-01,
         -5.87082825e-15,  3.23442301e-01, -1.74186519e-14,
          1.38393701e-14, -6.35254537e-01, -4.18965733e-14,
         -1.55100288e+00],
        [-1.62697034e-03,  4.67566346e-01, -1.93990743e-16,
         -2.18105724e-01,  1.23267514e-1

In [51]:
mf.mo_energy

NPArrayWithTag([-2.61436357e+02, -3.20149314e+01, -2.74971510e+01,
                -2.74962281e+01, -2.74948798e+01, -4.23175066e+00,
                -2.80953413e+00, -2.80764756e+00, -2.80612339e+00,
                -6.34650869e-01, -6.32992638e-01, -2.18591243e-01,
                -2.18586241e-01, -1.79862596e-01, -3.45633195e-02,
                 3.38735889e-02,  3.42321749e-02,  1.63939819e-01,
                 1.73877158e-01,  4.02041685e-01,  4.10869493e-01,
                 4.11758450e-01,  1.70301994e+00,  1.70454539e+00,
                 1.75447464e+00,  1.75468810e+00,  1.82905015e+00])

In [44]:
hf = RelForte(mol, complex=False, verbose=True, density_fitting=False, decontract=False)
hf.run_rhf(mf_in=mc, transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(7,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
# hf.cumulants = make_cumulants(hf.rdms)
# hf.run_dsrg_mrpt3(s=0.5, relativistic=False, relax=None)
hf.run_mr_ldsrg2(s=0.5, relativistic=False, relax=None, maxiter=50, max_ncomm=12)

              PySCF RHF interface              
PySCF RHF time:                    0.0000467 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0087950 s
-----------------------------------------------
RHF time:                          0.0089207 s

          Non-relativistic CASCI(7,8)          
Will now allocate 0.001 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0006070 s
Time taken for 2-RDM build:        0.0016437 s
Time taken for 3-RDM build:        0.0215979 s
E_frzc:                          -75.4775474 Eh
E_cas:                           -23.8943145 Eh
E_nuc:                             0.0000000 Eh
E_casci:                         -99.3718619 Eh
E0 (from RDM):                   -99.3718619 Eh
Ecorr:                             0.0000000 Eh

Timing summary
... Hamil build:                    0.0040519 s
... Hamil diag:                     0.0000722 s
... RDM build:                      0.5565

In [45]:
hbar2_1b = hf.hbar_1b
hbar2_2b = hf.hbar_2b

In [38]:
hbar2_1b = hf.fock
hbar2_2b = hf.rhf_eri_full_asym

In [43]:
hf.rhf_eri_full_asym[5,6,5,6]

(0.23340754445141876+0j)

In [10]:
hbar2_1b = 1*(hf.H0A2_1b + 0.5*hf.Htilde1A1_1b + hf.H0A1_1b) + hf.F0 + hf.F1
hbar2_2b = 1*(hf.H0A2_2b + 0.5*hf.Htilde1A1_2b + hf.H0A1_2b) + hf.rhf_eri_full_asym

In [46]:
hbar_eom = np.zeros((hf.nhole,hf.nhole))
hbar_eom[hf.core,hf.core] = -hbar2_1b[hf.core,hf.core]
hbar_eom[hf.core,hf.ha] = -np.einsum('mu,uv->mv',hbar2_1b[hf.core,hf.active],hf.cumulants['gamma1']) \
                        + 0.5*np.einsum('mwux,uxwv->mv',hbar2_2b[hf.core,hf.active,hf.active,hf.active],hf.cumulants['lambda2'])
hbar_eom[hf.ha,hf.core] = hbar_eom[hf.core,hf.ha].T
hbar_eom[hf.ha,hf.ha] = \
    - np.einsum('vu,ux,wv->wx',hbar2_1b[hf.active,hf.active],hf.cumulants['gamma1'],hf.cumulants['gamma1']) \
    + np.einsum('vu,uwvx->wx',hbar2_1b[hf.active,hf.active],hf.cumulants['lambda2']) \
    + 0.5*np.einsum('yzuv,wy,uvzx->wx',hbar2_2b[hf.active,hf.active,hf.active,hf.active],hf.cumulants['gamma1'],hf.cumulants['lambda2']) \
    - 0.5*np.einsum('yzuv,vx,uwyz->wx',hbar2_2b[hf.active,hf.active,hf.active,hf.active],hf.cumulants['gamma1'],hf.cumulants['lambda2']) \
    + 0.25*np.einsum('yzuv,uvwyzx->wx',hbar2_2b[hf.active,hf.active,hf.active,hf.active],hf.cumulants['lambda3'])

In [61]:
hbar_ea_eom = np.zeros((hf.npart,hf.npart))
hbar_ea_eom[hf.pa,hf.pa] += np.einsum('vu,uw,xv->wx',hbar2_1b[hf.active,hf.active],hf.cumulants['eta1'],hf.cumulants['eta1']) \
    - np.einsum('vu,uxvw->wx',hbar2_1b[hf.active,hf.active],hf.cumulants['lambda2']) \
    - 0.5*np.einsum('wxuv,zx,uvwy->yz',hbar2_2b[hf.active,hf.active,hf.active,hf.active],hf.cumulants['eta1'],hf.cumulants['lambda2']) \
    + 0.5*np.einsum('wxuv,uy,vzwx->yz',hbar2_2b[hf.active,hf.active,hf.active,hf.active],hf.cumulants['eta1'],hf.cumulants['lambda2']) \
    - 0.25*np.einsum('wxuv,uvzwxy->yz',hbar2_2b[hf.active,hf.active,hf.active,hf.active],hf.cumulants['lambda3'])
hbar_ea_eom[hf.pa,hf.pv] += np.einsum('eu,uv->ve',hbar2_1b[hf.virt,hf.active],hf.cumulants['eta1']) \
    - 0.5*np.einsum('ewuv,uvwx->xe',hbar2_2b[hf.virt,hf.active,hf.active,hf.active],hf.cumulants['lambda2'])
hbar_ea_eom[hf.pv,hf.pa] = hbar_ea_eom[hf.pa,hf.pv].T
hbar_ea_eom[hf.pv,hf.pv] += hbar2_1b[hf.virt,hf.virt]

In [47]:
ovlp = np.eye(hf.nhole)
ovlp[hf.active,hf.active] = hf.cumulants['gamma1']

In [62]:
ovlp = np.eye(hf.npart)
ovlp[hf.pa,hf.pa] = hf.cumulants['eta1']

In [63]:
eigh_gen(hbar_ea_eom, ovlp)

EighResult(eigenvalues=array([-0.04133857,  1.33035538,  1.40196104,  1.40196104,  1.425765  ,
        1.425765  ,  1.50723642,  1.72582975,  1.78378012,  3.82980131,
        3.85390261,  3.85390261,  3.92693064,  3.92693064,  3.94590275,
        3.94590275,  3.98542581,  3.98542581,  3.99884968]), eigenvectors=array([[ 9.99216450e-01,  2.10178386e-17, -3.96646526e-14,
         4.66346847e-14, -1.50039748e-14,  1.33552065e-14,
         3.95788578e-02,  3.61647969e-17,  1.38945886e-17,
        -5.06194468e-20,  2.49319552e-20,  1.31543606e-21,
         6.18635956e-23,  4.16591066e-22,  7.82422832e-20,
        -5.09617970e-19, -5.13075188e-23, -1.26208179e-18,
         2.95787830e-19],
       [ 1.04083409e-16, -1.00000000e+00,  2.10552009e-14,
        -8.57892518e-14,  1.09432144e-14,  4.62714451e-14,
        -2.22044605e-15, -2.88844626e-16, -8.24468540e-16,
        -2.49159801e-17, -8.55458720e-17,  3.32033063e-17,
        -1.26214990e-19, -3.13522284e-17, -1.27741409e-16,
         1.1

In [34]:
eta = 1e-9
sevals, sevecs = np.linalg.eigh(ovlp)
trunc_indices = np.where(np.abs(sevals) > eta)[0]

In [64]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
C 0 0 0
''',
    basis = 'cc-pvdz', spin=, charge=0, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
cc = pyscf.cc.CCSD(mf)
cc.kernel()
e_ip, c_ip = cc.eaccsd(nroots=1)
print(e_ip)


WARN: RCCSD method does not support ROHF method. ROHF object is converted to UHF object and UCCSD method is called.



/Users/brianz98/miniforge3/envs/pyscf/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


-0.037680648546445206


In [66]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
F 0 0 0
''',
    basis = 'cc-pvdz', spin=1, charge=0, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
cisolver = pyscf.fci.FCI(mf)
e_fci_neut = cisolver.kernel()[0]
print('E(FCI) = %.12f' % cisolver.kernel()[0])

mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
F 0 0 0
''',
    basis = 'cc-pvdz', spin=0, charge=-1, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
cisolver = pyscf.fci.FCI(mf)
e_fci_cat = cisolver.kernel()[0]
print('E(FCI) = %.12f' % cisolver.kernel()[0])
print('delta E(FCI) = %.12f' % (e_fci_cat - e_fci_neut))

E(FCI) = -99.529518212126
E(FCI) = -99.560774035999
delta E(FCI) = -0.031255823872


In [3]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
C 0 0 0
C 1.2 0 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry='d2h'
)
mf = pyscf.scf.ROHF(mol)
mf.kernel()
mc = pyscf.mcscf.CASSCF(mf, 8, 8)
mc.kernel()

(-75.61685394878796,
 -17.953495615709272,
 FCIvector([[ 8.32425716e-01,  0.00000000e+00,  0.00000000e+00, ...,
              0.00000000e+00,  0.00000000e+00, -2.76213484e-04],
            [ 0.00000000e+00, -1.79541740e-02,  0.00000000e+00, ...,
              0.00000000e+00,  1.92525866e-03,  0.00000000e+00],
            [ 0.00000000e+00,  0.00000000e+00, -1.79541740e-02, ...,
             -1.92525866e-03,  0.00000000e+00,  0.00000000e+00],
            ...,
            [ 0.00000000e+00,  0.00000000e+00, -1.92525866e-03, ...,
             -2.96603755e-04,  0.00000000e+00,  0.00000000e+00],
            [ 0.00000000e+00,  1.92525866e-03,  0.00000000e+00, ...,
              0.00000000e+00, -2.96603755e-04,  0.00000000e+00],
            [-2.76213484e-04,  0.00000000e+00,  0.00000000e+00, ...,
              0.00000000e+00,  0.00000000e+00,  1.39973131e-03]]),
 NPArrayWithTag([[ 7.07378079e-01,  7.08842215e-01, -1.40229154e-02,
                   1.37864339e-02,  0.00000000e+00,  0.00000000e+

In [4]:
mc.fcisolver.wfnsym

0

In [45]:
mf.mol.

['Ag', 'B1g', 'B2g', 'B3g', 'B1u', 'B2u', 'B3u']

In [1]:
from relforte import *
import pyscf.cc, pyscf.mcscf

mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
N 0 0 0
N 0 0 1.1
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
# print(mf.e_tot)
mc = pyscf.mcscf.CASCI(mf, 6, 6)
mc.kernel()
dsrg = RelForte(mol, complex=False, verbose=True, density_fitting=False, decontract=False)
dsrg.run_rhf(mf_in=mc, transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
dsrg.run_casci(cas=(6,12), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
# dsrg.run_mr_ldsrg2(s=0.5, relativistic=False, relax=None, maxiter=50, max_ncomm=20)
dsrg.run_mr_ldsrg2(s=1.0, relativistic=False, relax=None, maxiter=50, e_tol=1e-10)
dsrg.run_ip_singles(pt=False)
print(dsrg.eom_eigvals)

              PySCF RHF interface              
PySCF RHF time:                    0.0001299 s
-----------------------------------------------
Building integrals...
Integral build time:               0.1021261 s
-----------------------------------------------
RHF time:                          0.1024840 s

         Non-relativistic CASCI(6,12)          
Will now allocate 13.660 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.6589320 s
Time taken for 2-RDM build:        4.1323597 s
Time taken for 3-RDM build:       36.6125569 s
E_frzc:                         -121.1298336 Eh
E_cas:                           -11.4645108 Eh
E_nuc:                            23.5724394 Eh
E_casci:                        -109.0219050 Eh
E0 (from RDM):                  -109.0219050 Eh
Ecorr:                            -0.0000000 Eh

Timing summary
... Hamil build:                    2.4494951 s
... Hamil diag:                     0.1709251 s
... RDM build:                     42.073

KeyboardInterrupt: 

In [7]:
from relforte import *
import pyscf.cc, pyscf.mcscf

mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
C 0 0 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
# print(mf.e_tot)
mc = pyscf.mcscf.CASSCF(mf, 4, 4)
mc = mc.state_average_(np.array([1]*9)/9)
# mc.fcisolver.nroots = 9
# mc.fcisolver.spin = 1
# mc.fix_spin_(ss=2)
res = mc.kernel()
dsrg = RelForte(mol, complex=False, verbose=True, density_fitting=False, decontract=False)
dsrg.run_rhf(mf_in=mc, transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
dsrg.run_casci(cas=(4,8), do_fci=False, rdm_level=3, relativistic=False, semi_canonicalize=True)
# dsrg.run_mr_ldsrg2(s=0.5, relativistic=False, relax=None, maxiter=50, max_ncomm=20)
dsrg.run_mr_ldsrg2(s=0.5, relativistic=False, relax='twice', maxiter=50, e_tol=1e-10)
dsrg.run_ip_singles(pt=False)
print(dsrg.eom_eigvals)

              PySCF RHF interface              
PySCF RHF time:                    0.0001242 s
-----------------------------------------------
Building integrals...
Integral build time:               0.0046251 s
-----------------------------------------------
RHF time:                          0.0047872 s

          Non-relativistic CASCI(4,8)          
Will now allocate 0.078 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0123281 s
Time taken for 1-RDM build:        0.0123222 s
Time taken for 1-RDM build:        0.0125992 s
Time taken for 1-RDM build:        0.0122159 s
Time taken for 1-RDM build:        0.0122869 s
Time taken for 1-RDM build:        0.0122900 s
Time taken for 1-RDM build:        0.0122261 s
Time taken for 1-RDM build:        0.0122833 s
Time taken for 1-RDM build:        0.0122871 s
Time taken for 2-RDM build:        0.0582771 s
Time taken for 2-RDM build:        0.0568709 s
Time taken for 2-RDM build:        0.0551019 s
Time taken for 2-RD

In [4]:
dsrg.casci_eigvals

array([-2.57208271, -2.57208271, -2.57208261, -2.57208261, -2.57208261,
       -2.57208261, -2.46100736, -2.46100736, -2.46100736, -2.46100736,
       -2.46100727, -2.46100727, -2.46100727, -2.46100727, -2.46100727,
       -2.46100727, -2.46100727, -2.46100727, -2.31975388, -2.31975388,
       -2.31975386, -2.31975386, -2.31975386, -2.31975386, -2.31975381,
       -2.31975381, -2.31975381, -2.31975381, -2.248535  , -2.248535  ,
       -2.17968471, -2.17968471, -2.17968471, -2.17968471, -2.17968444,
       -2.17968444, -2.13739431, -2.13739431, -2.13739431, -2.13739431,
       -2.06617549, -2.06617549, -2.06617548, -2.06617548, -2.06617548,
       -2.06617548, -2.06617545, -2.06617545, -2.06617545, -2.06617545,
       -1.98691522, -1.98691522, -1.98691522, -1.98691522, -1.98691509,
       -1.98691509])

In [51]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
B 0 0 0
''',
    basis = 'cc-pvdz', spin=1, charge=0, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
fci = pyscf.fci.FCI(mf)
fci.spin = 1
fci.nroots = 15
fci.kernel()


(array([-24.5906296 , -24.5906296 , -24.5906296 , -24.46161667,
        -24.46161667, -24.46161667, -24.35770276, -24.35770276,
        -24.35770276, -24.35770276, -24.35770276, -24.28812884,
        -24.24085712, -24.24085712, -24.24085712]),
 [FCIvector([[-9.63881989e-01,  1.29827748e-16,  2.50755822e-19, ...,
              -3.79896633e-20, -1.89462181e-10,  4.77033176e-10],
             [ 1.92956262e-05,  1.53084720e-17, -1.40620941e-19, ...,
               5.03773223e-11,  5.88147239e-06, -5.79196658e-06],
             [-1.56532219e-17, -2.99055124e-06,  3.88240619e-08, ...,
              -1.05273724e-20,  1.30634626e-20,  1.05315435e-19],
             ...,
             [ 1.30752587e-20,  2.91006774e-11, -1.48636226e-12, ...,
               1.02414638e-22,  1.64447335e-20,  2.12347955e-20],
             [ 3.35163696e-20, -2.12308935e-11,  1.08440851e-12, ...,
               1.00512134e-20, -2.63032327e-20,  1.97923773e-20],
             [-1.69517512e-19,  6.53622042e-13, -3.4545557

In [49]:
fci.eci

-24.590629599935593

In [5]:
import pyscf.cc, pyscf.mcscf

mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
Be 0 0 0
H 0 1 2.5
H 0 1 -2.5
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry='c2v'
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
print(mf.e_tot)
ncore = {'A1':2}
ncas = {'A1':1,'B2':1}
mo = pyscf.mcscf.sort_mo_by_irrep(mc, mf.mo_coeff, ncas, ncore)
mc = pyscf.mcscf.CASSCF(mf, 2, 2)
mc.mc2step(mo)

-15.4797437429023


(-15.568094560303948,
 -1.0707579567098406,
 FCIvector([[-0.77927418,  0.        ],
            [ 0.        ,  0.62668313]]),
 NPArrayWithTag([[ 1.00075978e+00, -1.26485846e-03,  0.00000000e+00,
                  -2.34524021e-03,  0.00000000e+00,  9.11450731e-03,
                   0.00000000e+00, -6.27939446e-01,  0.00000000e+00,
                  -2.32854190e-01,  0.00000000e+00,  0.00000000e+00,
                   3.90680570e-03,  0.00000000e+00,  0.00000000e+00,
                   4.50179689e-02,  1.43140415e-01,  0.00000000e+00,
                   0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                   7.47342467e-03,  0.00000000e+00, -9.49825430e-02],
                 [ 2.42729342e-04,  4.71381909e-01,  0.00000000e+00,
                  -1.64923898e-01,  0.00000000e+00,  5.36699321e-02,
                   0.00000000e+00, -1.65255923e+00,  0.00000000e+00,
                  -5.90844758e-01,  0.00000000e+00,  0.00000000e+00,
                   1.15071351e-02,  0.0000000

In [4]:
import pyscf.cc, pyscf.mcscf

mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
Ne
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry='d2h'
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
print(mf.e_tot)
cc = pyscf.cc.CCSD(mf)
cc.frozen = 1
cc.kernel()
print(cc.e_tot)

-128.48877555174084
-128.67779226924011


In [2]:
mc.e_tot

-37.70108663196813

In [23]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
He 0 0 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
cisolver = pyscf.fci.FCI(mf)
e_fci_neut = cisolver.kernel()[0]
print('E(FCI) = %.12f' % cisolver.kernel()[0])

cc = pyscf.cc.CCSD(mf)
cc.kernel()
e_ip, c_ip = cc.eaccsd(nroots=1)
print(e_ip)

mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
He 0 0 0
''',
    basis = 'cc-pvdz', spin=1, charge=-1, symmetry=False
)
mf = pyscf.scf.RHF(mol)
mf.kernel()
cisolver = pyscf.fci.FCI(mf)
e_fci_cat = cisolver.kernel()[0]
print('E(FCI) = %.12f' % cisolver.kernel()[0])
print('delta E(FCI) = %.12f' % (e_fci_cat - e_fci_neut))

E(FCI) = -2.887594831091
1.372614561270915
E(FCI) = -1.514685677280
delta E(FCI) = 1.372909153810


In [13]:
eri = mol.intor('int2e',aosym='s1')
mo1 = np.random.random((mol.nao_nr(), 10))
eri1 = pyscf.ao2mo.incore.full(eri, mo1)


In [15]:
mol.intor('int1e_kin') + mol.intor('int1e_nuc')

array([[-2.44268625e+01,  5.93468752e+00, -4.30007168e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.11022302e-16,
         0.00000000e+00,  0.00000000e+00],
       [ 5.93468752e+00, -7.01647662e+00, -3.23389503e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-4.30007168e+00, -3.23389503e+00, -4.95902285e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -5.62906904e+00,  0.00000000e+00,  0.00000000e+00,
        -2.57802418e+00,  0.00000000e+00,  0.

In [17]:
mf.get_hcore()

array([[-2.44268625e+01,  5.93468752e+00, -4.30007168e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.11022302e-16,
         0.00000000e+00,  0.00000000e+00],
       [ 5.93468752e+00, -7.01647662e+00, -3.23389503e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-4.30007168e+00, -3.23389503e+00, -4.95902285e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -5.62906904e+00,  0.00000000e+00,  0.00000000e+00,
        -2.57802418e+00,  0.00000000e+00,  0.